# Configure LaminDB on the command line with `lndb`

```{note}

Everything here is tested during CI except for the very first time a user calls `lndb set`:
The code executed in that case merely asks for email confirmation.

```

In [ ]:
!lndb --help

In [ ]:
from lamindb import load_settings

## Sign up

### The user does not yet exist

You only need to sign up a single time! For a non-signed-up email, call
```
!lndb signup --email raspbear@gmx.de
```
which prints
```
Please *confirm* the sign-up email. After that, proceed to `lndb init`!

Generated login secret: O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR.
Email & secret persist in: /Users/falexwolf/.lndb/lndb.env.
Going forward, credentials are auto-loaded. In case of loss, you can always recover your secret via email.
```

### The user confirmed the email and hence, is signed up

If the user tries to sign up again, an error message is raised:

In [ ]:
!lndb signup --email raspbear@gmx.de

### The user did not confim the email and tries to sign up again

This raises the following error. It was a bit tricky to implement, see {doc}`../notes/multiple-sign-ups-same-email`.

```
  File "/opt/anaconda3/envs/base1/bin/lndb", line 8, in <module>
    sys.exit(main())
  File "/Users/falexwolf/repos/lamindb/lamindb/__main__.py", line 28, in main
    return _setup.sign_up_first_time(
  File "/Users/falexwolf/repos/lamindb/lamindb/_setup/_setup.py", line 40, in sign_up_first_time
    secret = sign_up_hub(email)
  File "/Users/falexwolf/repos/lamindb/lamindb/_setup/_hub.py", line 34, in sign_up_hub
    raise RuntimeError(
RuntimeError: It seems you already signed up with this email. Please click on the link in the confirmation email that you should have received from lamin.ai.
```

Depending on timing, it might also raise a `429 Too Many Requests` error, which is fine, too.

## Login

In [ ]:
!lndb login --email "raspbear@gmx.de" --secret "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"

## Configure LaminDB instance

### Configure with local storage

```{note}

After the first time sign up, the following prints an additional first line as below.

```

```
Completing user sign up. Only happens once!
Created lndb instance: mydata/mydata.lndb
```

In [ ]:
!lndb init --storage "mydata"

In [ ]:
settings = load_settings()

In [ ]:
settings

In [ ]:
settings.instance_name

In [ ]:
assert settings.cloud_storage == False
assert settings.instance_name == "mydata"
assert settings.user_secret == "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"
assert settings.user_id == "9ypQ1yrW"
assert settings.storage_dir.as_posix() == "mydata"
assert settings.cache_dir is None
assert settings.db == "sqlite:///mydata/mydata.lndb"
assert settings.user_email == "raspbear@gmx.de"

Remove the test directory:

In [ ]:
settings._sqlite_file.unlink()
settings.storage_dir.rmdir()

### Configure with cloud storage

In [ ]:
!lndb init --storage "s3://lamin0"

In [ ]:
settings = load_settings()

In [ ]:
settings

In [ ]:
settings.instance_name

In [ ]:
settings.cache_dir

In [ ]:
settings.db

In [ ]:
settings._sqlite_file

In [ ]:
settings._sqlite_file_local

In [ ]:
assert settings.cloud_storage == True
assert settings.user_secret == "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"
assert settings.user_id == "9ypQ1yrW"
assert str(settings.storage_dir) == "s3://lamin0"
assert str(settings._sqlite_file) == "s3://lamin0/lamin0.lndb"
assert settings.user_email == "raspbear@gmx.de"

Clean up the test case.

In [ ]:
settings._sqlite_file.unlink()
settings._sqlite_file_local.unlink()

## Edge cases

### Log in with in-sufficient information

Log in without email and secret (why do we allow this??), loading it from lndb.env:

In [ ]:
!lndb login

In [ ]:
from lamindb._setup._settings_store import settings_file

In [ ]:
settings_file

In [ ]:
settings_file.unlink()

If we remove the settings file and then try to log in, we get the following error:

In [ ]:
!lndb login

If we add an email, but still don't have a secret, we get a different error:

In [ ]:
!lndb login --email raspbear@gmx.de

Now we're all good again! 😅

In [ ]:
!lndb login --email raspbear@gmx.de --secret "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"

### Configuration with insufficient information

In [ ]:
settings_file.unlink()

We'd also get an error about email if we call config without it being set:

In [ ]:
!lndb init --storage "mydata"

Let's set it, now, but not provide a storage location:

In [ ]:
!lndb login --email raspbear@gmx.de --secret "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"

In [ ]:
!lndb init

However, if we set a location:

In [ ]:
!lndb init --storage "mydata"

We can now just load it:

In [ ]:
!lndb init

### Invalid command

In [ ]:
!lndb invalid

### Future to-be-implemented Postgres

Configuration for a multi-storage postgres cloud solution would be:

In [ ]:
!lndb init --db "instance_name,postgres_url"

Switch back.

In [ ]:
!lndb init --db "sqlite"